In [96]:
import matplotlib
import re
import os
import sys

##Current Step: Start plotting connection indexes by IP Header Size
##Plot connections that use most bandwidth -- using IP Header averages. (Top 10 for game connections etc)

In [97]:
capture_src = 'captures'
capture_dst = 'captures_split'
processed = 'processed_log.txt'
packet_match_regex = r'.*IP.*proto\W(?P<proto>\w+).*length\W(?P<ipLength>\d+).*\r?\n\W*(?P<srcHost>[\w\.\-]+)\.(?P<srcPort>[\w]+)\W\>\W(?P<dstHost>[\w\.\-]+)\.(?P<dstPort>[\w]+).*Flags.*length\W(?P<tcpLength>\d+).*'

In [98]:
class connection_map:
    connection_list = list()
    def __init__(self):
        self.connection_list = list()
    def get_connection_index(self,packet):
        connection_string = "%s:%s,%s:%s,%s"%(packet.src_host,packet.src_port,packet.dst_host,packet.dst_port,packet.proto)
        try:
            connection_index = self.connection_list.index(connection_string)
            return connection_index
        except ValueError:
            self.connection_list.append(connection_string)
            return self.connection_list.index(connection_string)
    def write_connection_map_to_file(self,file):
        with open(file,'w') as connection_map_file:
            for idx,connection_string in enumerate(self.connection_list):
                connection_map_file.write(str(idx)+","+connection_string+'\n')


In [99]:
class packet:
    packet_string = ""
    src_host = ""
    src_port = ""
    dst_host = ""
    dst_port = ""
    proto = ""
    ip_length = 0
    tcp_length = 0
    def __init__(self,src_host,src_port,dst_host,dst_port,proto,packet_string,ip_length,tcp_length):
        self.src_host = src_host
        self.src_port = src_port
        self.dst_host = dst_host
        self.dst_port = dst_port
        self.proto = proto
        self.packet_string = packet_string
        self.ip_length = ip_length
        self.tcp_length = tcp_length
    def to_string(self):
        print("srcHost:%s\nsrcPort:%s\ndstHost:%s\ndstPort:%s\nproto:%s\n" % (self.src_host,self.src_port,self.dst_host,self.dst_port,self.proto))

In [100]:
def split_packet_flows(capture_file,split_dir,base_filename):
    line_one = ''
    line_two = ''
    packet_connection_map = connection_map()
    
    #Ensure that the split flows directory exists
    try:
        os.mkdir(split_dir)
    except FileExistsError:
        pass #We don't care if the directory already exists    
    
    #Loop through the capture file searching for normal packets, ignore everything else.
    with open(capture_file,'r') as src:
        for line in src:
            line_one = line_two
            line_two = line
            line_pair = line_one + line_two
            
            packet_match = re.match(packet_match_regex,line_pair,re.MULTILINE)
            if(packet_match):
                cur_packet = packet(packet_match.group('srcHost'),packet_match.group('srcPort'),packet_match.group('dstHost'),packet_match.group('dstPort'),packet_match.group('proto'),line_pair,packet_match.group('ipLength'),packet_match.group('tcpLength'))
                connection_idx = packet_connection_map.get_connection_index(cur_packet)
                
                split_file_path = split_dir + os.sep + base_filename + '.' + str(connection_idx) + '.cap'
                #Write packet to file with connection index
                with open(split_file_path,'a') as split_dst:
                    split_dst.write(cur_packet.packet_string+os.linesep)
    packet_connection_map.write_connection_map_to_file(split_dir+os.sep+'connections.map')
def get_packets_in_file(capture_file):
    line_one = ''
    line_two = ''
    packets = list()
    with open(capture_file,'r') as src:
        for line in src:
            line_one = line_two
            line_two = line
            line_pair = line_one + line_two
            
            packet_match = re.match(packet_match_regex,line_pair,re.MULTILINE)
            if(packet_match):
                cur_packet = packet(packet_match.group('srcHost'),packet_match.group('srcPort'),packet_match.group('dstHost'),packet_match.group('dstPort'),packet_match.group('proto'),line_pair,packet_match.group('ipLength'),packet_match.group('tcpLength'))
                packets.append(cur_packet)
    return packets

In [101]:
try:
    for x in os.walk(capture_src): #each os.walk element is: dirpath, subdir-names, dir-filenames
        outdir = x[0].replace(capture_src,capture_dst)
        try:
            os.mkdir(outdir)
        except FileExistsError:
            pass #We don't care if the directory already exists
        for capture in x[2]:
            if capture.endswith(".cap"):
                base_file = capture[:-4]
                date_file = base_file + '.date'
                src_file = x[0]+os.sep+capture
                split_dir = (x[0]+os.sep+base_file).replace('captures','captures_split')
                print(src_file)
                print(split_dir)
                print(base_file)
                split_packet_flows(src_file,split_dir,base_file)
                raise KeyboardInterrupt
                '''
                capture_packets = get_packets_in_file(x[0]+"\\"+capture)
                
                for packet in capture_packets:
                    print(packet)
                    raise KeyboardInterrupt
                '''
                        
except KeyboardInterrupt:
    pass
            
    #print(outdir)

captures\games\hon\hon_raw_capture10.cap
captures_split\games\hon\hon_raw_capture10
hon_raw_capture10


In [53]:
#Isolate the outputs into <src_name>.connectionNumber.cap
##Then plot the connections onto a single graph.
###The dominant connection in a packet capture is the one you keep